# How to interactively inspect Target Pixel Files and Light Curves?

`.interact()` offers instantaneous interactive selection of the pixel mask, and instantaneous generation of the resulting lightcurve.  You can click on individual pixels and the aperture photometry seamlessly updates.  The mask can be defined with either individual clicking of pixels, or clicking and dragging a box over a rectangular set of pixels.  *De*selection of individual pixels also works-- simply re-click a pixel that you wish to take away from your mask.  Finally, you can save your mask and lightcurve as a fits file by clicking on the `Save Lightcurve` button.

The default mask is the Kepler "pipeline" mask, and can be modified as described above.

In this tutorial we introduce and explain the `tpf.interact()` tool that accomplishes the goal of interactively inspecting the TPFs and lightcurve simultaneously.

### The  `.interact()` method applies to target pixel files.

Using interact should be as simple as downloading a Kepler Target Pixel File (TPF) and running the method `.interact()`.  This method can only be run in a Jupyter Notebook at the moment, and is limited to lightcurves with fewer than 30000 cadences.

Let's first look at the target HL Tau, a young star that possesses a gapped circumstellar disk [imaged by the Atacama Large Millimeter Array](http://www.almaobservatory.org/en/press-release/revolutionary-alma-image-reveals-planetary-genesis/).

In [1]:
from lightkurve import search_targetpixelfile
tpf = search_targetpixelfile("HL Tau", mission='K2', campaign=13).download()

The K2 postage stamp of HL Tau contains a portion of a nearby source of comparable brightness.  The weakly overlapping point spread functions (PSFs) of these sources motivate some caution in aperture choice.  Let's interactively assign a custom aperture photometry pixel mask:

```python
tpf.interact()
```

![Lightkurve interact() demo](images/20180924_interact_HLTau.gif)

You can move the large bottom left slider to change the location of the verical red bar, which indicates which cadence is being shown in the TPF postage stamp image.  The slider beneath the TPF postage stamp image controls the screen stretch, which defaults to logarithmic scaling initialized to 1% and 95% lower and upper limits respectively.

You can move your cursor over individual data points to show hover-over tool-tips indicating additional information about that datum.  Currently the tool tips list the cadence, time, flux, and quality flags.  The tools on the right hand side of the plots enable zooming, and pixel selection.

We see that the starting mask (the Kepler pipeline mask, by default), shows huge jumps in flux between times 3000 and 3020.  These jagged artifacts disappear upon the selection of a larger aperture-- large enough to encompass most of the point spread function of the star.  The end result shows a time-series lightcurve of a young disk-bearing star.

Interaction modes:

- Clicking on a single pixel shows the time-series lightcurve of that pixel alone.  
- `shift`-clicking on multiple pixels shows the lightcurve using that pixel mask.
- `shift`-clicking on an already-selected pixel will *de*-select that pixel.
- Clicking and dragging a box will make a rectangular aperture mask-- individual pixels can be deselected from this mask by shift-clicking (box de-selecting does not work).
- The screen stretch high and low limits can be changed independently by clicking and dragging each end, or simultaneously by clicking and dragging in the middle.
- The cadence slider updates the postage stamp image at the position of the vertical red bar in the lightcurve.
- Clicking on a position in the lightcurve automatically seeks to that Cadence Number.
- The left and right arrows can be clicked to increment the cadence number by one.

The `interact()` tool works for both Kepler or K2 data.  KOI 6.01 (KIC 3248033) sits 4 pixels away from eclipsing binary KOI 1759.01 (KIC 3248019).  An unwise choice of pixels can give rise to a spurious exoplanet signal.

In [2]:
tpf = search_targetpixelfile('KIC 3248033', quarter=4).download()

```python
tpf.interact()
```

![Lightkurve interact() demo](images/20180925_interact_EB_contam.gif)

You can see that the Kepler pointing is remarkably stable over this Kepler quarter of 89 days.  The value of interact arises from its ability to discern the spatial origin of signals.  In this case, an eclipsing binary occupies the pixels towards the top of the postage stamp image.  The target of interest occupies the pixels in the middle.  The optimal aperture mask should avoid pixels near the top in order to mitigate an artificial planet signal.

### Troubleshooting `.interact()`

There are a few known limitations of `interact()`.  First, interact only works in a Jupyter Notebook.  We are experimenting with solutions to allow interact to work from additional settings.  Second, you must either run interact from the default Jupyter notebook address "localhost:8888", **or tell interact the name of the notebook server**, for example:

```python
tpf.interact(notebook_url='localhost:8893')
```

otherwise, you will likely see an error message like this:

In [3]:
tpf.interact(notebook_url='localhost:4321')

ERROR:bokeh.server.views.ws:Refusing websocket connection from Origin 'http://localhost:8888';                       use --allow-websocket-origin=localhost:8888 or set BOKEH_ALLOW_WS_ORIGIN=localhost:8888 to permit this; currently we allow origins {'localhost:4321'}


Other unexpected behaviors can occur in interact.  For example, when resetting the image views the revised plot scaling may not display the data. Attempting to de-select with a box will cause unexpected toggling of pixels.  De-selection only works with the tap tool, so box-selections should use caution not to overlap with existing pixels.  Surprisingly, de-selection does not work for the most-recently added pixel.  Lightcurves with more than 30,000 cadences require either an increase in the `max_cadences` kwarg, or slicing of your input TPF like so:

```python
tpf[0:1000,].interact()
tpf.interact(max_cadences=40000)
```

We are continuing to improve interact.  We welcome any feedback about the tool or feature requests.  You can [email us](https://keplerscience.arc.nasa.gov/helpdesk.html) or [open an Issue on our lightkurve GitHub repository](https://github.com/KeplerGO/lightkurve/issues).  


Finally, we'd like to thank the developers of [Bokeh](https://bokeh.pydata.org/en/latest/), upon which interact relies.  